# Agent模块代码实战

 核心思想是ReAct 推理+操作

联网API获取：https://app.tavily.com/home

准备工作：
pip install --upgrade langchain-openai tavily-python
提前准备tavily_api_key: 到Tavily 官方网站注册获取API KEY
Tavily Search 是一个专为人工智能体设计的搜索API，主要用于实时在线信息的集成。
Tavily Search API 允许人工智能开发人员将他们的应用程序与互联网上的实时信息结合起来，这对于需要获取最新数据或执行复杂查询的AI应用来说非常有用。通过这个API，AI可以访问大量的在线信息，并以高效的方式检索和处理数据，从而为用户提供更准确、更及时的信息或服务。例如，如果代理需要回答一个大模型未训练的涉及最新事件的问题，它可以通过Tavily Search API联网来获取相关
信息，然后将这些信息整合到其回答中。


1.使用TavilySearchResults ，则需要提前到Tavily 官方网站注册获取API KEY
2.使用SerpAPIWrapper 使用的是谷歌的搜索引擎，需要到https://serpapi.com/manage-api-key申请Google的API KEY


## 1 Self-ask with search 自我询问类型的Agent

### 1.联网搜索方式1： Tavily API（搜索API） + Fireworks APi （开源模型）
下面样例中展示，Agent通过自我询问的方式查询互联网API获取答案
准备工作：
1.下面使用到了搜索引擎Tavily，需要到Tavily官网申请的API KEY
2.下面使用到了Fireworks的模型需要到Fireworks.ai申请API KEY 配置到环境变量

In [2]:
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.llms import Fireworks
from langchain_community.tools.tavily_search import TavilyAnswer

#需要申请搜索引擎Tavily的API KEY  （Tavily 专门为人工智能体设计了一款搜索 API）
tools = [TavilyAnswer(max_results=1, name="Intermediate Answer",tavily_api_key='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')]
# 获取prompt
prompt = hub.pull("hwchase17/self-ask-with-search")

# 选择大模型 需要申请Fireworks.ai的API KEY （Fireworks.ai 是一家提供开源模型API服务）
llm = Fireworks(fireworks_api_key=os.environ['FIREWORKS_API_KEY'])

#构建代理
agent = create_self_ask_with_search_agent(llm, tools, prompt)
#创建代理执行器
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {"input": "上个赛季的NBA总冠军是哪支球队？"}
)

D:\Program Files (x86)\python3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.fireworks.Fireworks` was deprecated in langchain-community 0.0.26 and will be removed in 0.2. An updated version of the class exists in the langchain-fireworks package and should be used instead. To use it run `pip install -U langchain-fireworks` and import as `from langchain_fireworks import Fireworks`.
  warn_deprecated(




> Entering new AgentExecutor chain...
 Yes.
Follow up: Which NBA team won the championship last season?According to the data provided, the Denver Nuggets won the NBA championship last season in 2022.
So the final answer is: Denver Nuggets

> Finished chain.


{'input': '上个赛季的NBA总冠军是哪支球队？', 'output': 'Denver Nuggets'}

### 2.联网搜索方式二： Google API（搜索API） + OpenAI APi （开源模型）
下面样例中展示，Agent通过自我询问的方式查询互联网API获取答案
准备工作：
1.下面使用到了搜索引擎google，需要到google官网申请的API KEY
https://developers.google.com/?hl=zh-cn

In [2]:
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.tools.google_search import GoogleSearchResults
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)
#需要申请对应搜索引擎的API KEY ;  https://serpapi.com/manage-api-key
#google_api_key=os.environ['SERPAPI_API_KEY']
tools = [GoogleSearchResults(google_cse_id=111111,google_api_key=os.environ['SERPAPI_API_KEY'])]
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(llm, tools, prompt)

# 创建代理执行器
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "谁是现任美国总统?"})

ValidationError: 1 validation error for GoogleSearchResults
api_wrapper
  field required (type=value_error.missing)

### 3.联网搜索方式三： Tavily API（搜索API） + OpenAI APi （开源模型）
下面样例中展示，Agent通过自我询问的方式查询互联网API获取答案
准备工作：
1.下面使用到了搜索引擎Tavily，需要到Tavily官网申请的API KEY

In [1]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

#需要申请搜索引擎Tavily的API KEY
tools = [TavilySearchResults(max_results=1,tavily_api_key= "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")]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")

# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "卡塔尔世界杯冠军球队是?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '卡塔尔世界杯冠军球队'}`


[{'url': 'https://baike.baidu.com/item/2022年卡塔尔世界杯决赛/62470695', 'content': '2022年卡塔尔世界杯决赛，于2022年12月18日在卢赛尔体育场举行，以决出2022年卡塔尔世界杯冠军归属，对阵双方为阿根廷与法国。最终阿根廷队以7:5(点球大战4:2)击败卫冕冠军法国队，时隔36年再次捧起大力神杯。'}]卡塔尔世界杯冠军球队是阿根廷队，他们在2022年的世界杯决赛中以点球大战4:2击败法国队，时隔36年再次捧起大力神杯。

> Finished chain.


{'input': '卡塔尔世界杯冠军球队是?',
 'output': '卡塔尔世界杯冠军球队是阿根廷队，他们在2022年的世界杯决赛中以点球大战4:2击败法国队，时隔36年再次捧起大力神杯。'}

## 2. OpenAI functions

### 2.1 OpenAI functions代理方式

In [2]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

#选择Tavily搜索引擎作为查询工具
tools = [TavilySearchResults(max_results=1)]

In [3]:
#选择提示词
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [4]:
# 选择OpenAI的模型驱动代理
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# 构造 OpenAI Functions agent
agent = create_openai_functions_agent(llm, tools, prompt)

In [5]:
# #创建代理执行器
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'LangChain'}`


[{'url': 'https://www.langchain.com/', 'content': 'Join Discord\nContribute on GitHub\n5.3M\nMonthly Downloads\n67k\nGithub Stars\n1800+\nContributors\n34k+\nApps Powered\nReady to build?\nGo To Docs\nGo To Docs\nGo To Docs\nContact Sales\nContact Sales\nContact Sales\nProducts and use-cases\nLangChain\nLangSmith\nRetrieval\nAgents\nInspiration\nCode\nGitHub\nLangChain Hub\nPython Docs\nJS/TS Docs\nSocial\nTwitter\nDiscord\nBlog\nLinkedIn\nYouTube\nTerms of Service\nSign up for our newsletter\nProducts and use-cases\nLangChain\nLangSmith\nRetrieval\nAgents\nInspiration\nCode\nGitHub\nLangChain Hub\nPython Docs\nJS/TS Docs\nSocial\nTwitter\nDiscord\nBlog\nLinkedIn\nYouTube\nTerms of Service\nSign up for our newsletter\nProducts and use-cases\nLangChain\nLangSmith\nRetrieval\nAgents\nInspiration\nCode\nGitHub\nLangChain Hub\nPython Docs\nJS/TS Docs\nSocial\nTwitter\nDiscord\nBl

{'input': 'what is LangChain?',
 'output': "LangChain is a platform that offers a complete set of powerful building blocks for building context-aware, reasoning applications using flexible abstractions and an AI-first toolkit. It enables developers to leverage new cognitive architectures and battle-tested orchestration to harness the power of non-deterministic models, particularly Large Language Models (LLMs). LangChain provides tools for chatbots, Q&A over docs, summarization, copilots, workflow automation, document analysis, and custom search. It is used by global corporations, startups, and developers to build advanced applications with LLMs. If you'd like to learn more, you can visit their website [here](https://www.langchain.com/)."}

In [6]:
### 使用聊天历史
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name?",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob.'}

## 3. 自定义Agent

### 3.1.定义LLM

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

### 3.2.自定义处理工具

In [11]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

In [12]:
tools = [get_word_length]

### 3.3.创建prompt

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are very powerful assistant, but don't know current events", ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

### 3.4.绑定工具到LLM

In [14]:
llm_with_tools = llm.bind_tools(tools)

### 3.5.创建Agent

In [15]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)


### 3.6.构建AgentExecutor 再调用执行

In [16]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
list(agent_executor.stream({"input": "How many letters in the word eudca"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


5The word "eudca" has 5 letters.

> Finished chain.


[{'actions': [OpenAIToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_AzluXlWFt8xvgOAZRuHJ3Vwq', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'})], tool_call_id='call_AzluXlWFt8xvgOAZRuHJ3Vwq')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_AzluXlWFt8xvgOAZRuHJ3Vwq', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_c

## 4. Stream流式操作

### 比如下面这道题：猫藏身的地方有哪些物品
需要2个工具，1.先找到猫藏身的地方，2.再确认藏身的地方有哪些物品

In [18]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)

In [19]:
import random


@tool
async def where_cat_is_hiding() -> str:
    """Where is the cat hiding right now?"""
    return random.choice(["under the bed", "on the shelf"])


@tool
async def get_items(place: str) -> str:
    """Use this tool to look up which items are in the given place."""
    if "bed" in place:  # For under the bed
        return "socks, shoes and dust bunnies"
    if "shelf" in place:  # For 'shelf'
        return "books, penciles and pictures"
    else:  # if the agent decides to ask about a different place
        return "cat snacks"

In [20]:
#方法调用测试
await where_cat_is_hiding.ainvoke({})

'under the bed'

In [21]:
#方法调用测试
await get_items.ainvoke({"place": "shelf"})

'books, penciles and pictures'

In [24]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
# print(prompt.messages) -- to see the prompt
tools = [get_items, where_cat_is_hiding]
agent = create_openai_tools_agent(
    model.with_config({"tags": ["agent_llm"]}), tools, prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools).with_config(
    {"run_name": "Agent"}
)

In [25]:
# Note: We use `pprint` to print only to depth 1, it makes it easier to see the output from a high level, before digging in.
import pprint

chunks = []
#异步地从agent_executor.astream()获取数据块
async for chunk in agent_executor.astream({"input": "what's items are located where the cat is hiding?"}):
    chunks.append(chunk)
    print("------")
    pprint.pprint(chunk, depth=1)

#它看起来像这样：
#动作输出actions
#观测输出steps
#动作输出actions
#观测输出steps
#...（继续直到达到目标）...
#然后，如果达到最终目标，智能体将输出最终答案。

------
{'actions': [...], 'messages': [...]}
------
{'messages': [...], 'steps': [...]}
------
{'actions': [...], 'messages': [...]}
------
{'messages': [...], 'steps': [...]}
------
{'messages': [...],
 'output': 'The items located where the cat is hiding (on the shelf) are '
           'books, pencils, and pictures.'}


In [26]:
for chunk in chunks:
    print(chunk["messages"])

[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_lCJicoLUxyNbVzzAkghNHu7o', 'function': {'arguments': '{}', 'name': 'where_cat_is_hiding'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'})]
[FunctionMessage(content='on the shelf', name='where_cat_is_hiding')]
[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_9jsmlG4dhjTKuQQotvZFJjrX', 'function': {'arguments': '{"place":"on the shelf"}', 'name': 'get_items'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'})]
[FunctionMessage(content='books, penciles and pictures', name='get_items')]
[AIMessage(content='The items located where the cat is hiding (on the shelf) are books, pencils, and pictures.')]


In [27]:
#查看整个处理流程中的行动、观察的过程
async for chunk in agent_executor.astream(
    {"input": "what's items are located where the cat is hiding?"}
):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("---")

Calling Tool: `where_cat_is_hiding` with input `{}`
---
Tool Result: `on the shelf`
---
Calling Tool: `get_items` with input `{'place': 'on the shelf'}`
---
Tool Result: `books, penciles and pictures`
---
Final Output: The items located where the cat is hiding (on the shelf) are books, pencils, and pictures.
---


In [28]:
#查看整个处理流程中的事件处理过程
async for event in agent_executor.astream_events(
    {"input": "where is the cat hiding? what items are in that location?"},
    version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

Starting agent: Agent with input: {'input': 'where is the cat hiding? what items are in that location?'}
--
Starting tool: where_cat_is_hiding with inputs: {}
Done tool: where_cat_is_hiding
Tool output was: on the shelf
--
--
Starting tool: get_items with inputs: {'place': 'on the shelf'}
Done tool: get_items
Tool output was: books, penciles and pictures
--
The| cat| is| hiding| on| the| shelf|.| In| that| location|,| you| can| find| books|,| pencils|,| and| pictures|.|
--
Done agent: Agent with output: The cat is hiding on the shelf. In that location, you can find books, pencils, and pictures.


## 5. iterator迭代操作

###查找3个质数，并计算他们的乘积
%pip install --upgrade --quiet  numexpr

In [32]:
from langchain.agents import AgentType, initialize_agent
from langchain.chains import LLMMathChain
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI

# 目前只有 GPT-4 才支持这个智能化的处理过程，知道最终需要调用计算器进行计算
llm = ChatOpenAI(temperature=0, model="gpt-4")
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

In [33]:
primes = {998: 7901, 999: 7907, 1000: 7919}


class CalculatorInput(BaseModel):
    question: str = Field()


class PrimeInput(BaseModel):
    n: int = Field()


def is_prime(n: int) -> bool:
    if n <= 1 or (n % 2 == 0 and n > 2):
        return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0:
            return False
    return True


def get_prime(n: int, primes: dict = primes) -> str:
    return str(primes.get(int(n)))


async def aget_prime(n: int, primes: dict = primes) -> str:
    return str(primes.get(int(n)))


tools = [
    Tool(
        name="GetPrime",
        func=get_prime,
        description="A tool that returns the `n`th prime number",
        args_schema=PrimeInput,
        coroutine=aget_prime,
    ),
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to compute mathematical expressions",
        args_schema=CalculatorInput,
        coroutine=llm_math_chain.arun,
    ),
]

In [34]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))

In [ ]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
question = "What is the product of the 998th, 999th and 1000th prime numbers?"

for step in agent_executor.iter({"input": question}):
    if output := step.get("intermediate_step"):
        action, value = output[0]
        if action.tool == "GetPrime":
            print(f"Checking whether {value} is prime...")
            assert is_prime(int(value))
        # Ask user if they want to continue
        _continue = input("Should the agent continue (Y/n)?:\n") or "Y"
        if _continue.lower() != "y":
            break